**Melatih dan Men-deploy Model TensorFlow dalam Skala Besar**

TF Serving diperkenalkan sebagai solusi yang sangat efisien dan teruji untuk melayani model. Ini dapat menangani beban tinggi, melayani berbagai versi model, dan secara otomatis menyebarkan versi terbaru dari sebuah repositori model. Bab ini juga membahas tentang penanganan versioning model, transisi antar model, rollback ke versi sebelumnya, dan menjalankan eksperimen A/B.

Pelatihan model yang memakan waktu lama juga menjadi perhatian utama. Untuk mempercepatnya, penggunaan hardware accelerators seperti GPU atau TPU disarankan. Selain itu, model dapat dilatih di beberapa mesin, masing-masing dilengkapi dengan beberapa hardware accelerators, yang difasilitasi oleh TensorFlow's Distribution Strategies API.

Bab ini melanjutkan dengan membahas cara men-deploy model ke TF Serving dan Google Cloud AI Platform. Kemudian, dibahas juga deployment model ke aplikasi seluler, perangkat embedded, dan aplikasi web. Terakhir, bab ini menjelaskan cara mempercepat komputasi menggunakan GPU dan cara melatih model di beberapa perangkat dan server menggunakan Distribution Strategies API.

**Penjelasan Teoritis**

Berikut adalah penjelasan teoritis dari poin-poin penting di Bab 19:

* **Serving a TensorFlow Model (Melayani Model TensorFlow)**
    Ketika infrastruktur aplikasi berkembang, lebih baik model TensorFlow dibungkus dalam sebuah layanan kecil yang tugas utamanya adalah membuat prediksi. Ini disebut "serving" model. Pendekatan ini memungkinkan model decoupled dari sisa infrastruktur, sehingga mudah untuk mengganti versi model, melakukan scaling layanan secara independen, dan melakukan eksperimen A/B.

    * **SavedModel Format (Format SavedModel)**
        Untuk men-deploy model ke TF Serving, model harus diekspor ke format SavedModel TensorFlow. SavedModel adalah format serialisasi universal untuk model TensorFlow, yang menyimpan computation graph model dan bobotnya. SavedModel disimpan sebagai direktori yang berisi file `saved_model.pb` (mendefinisikan computation graph) dan subdirektori `variables` (berisi nilai variabel). Model juga dapat menyertakan subdirektori `assets` untuk data tambahan seperti file kosakata. Penting untuk menyertakan semua lapisan pra-pemrosesan dalam model yang diekspor agar model dapat menerima data dalam bentuk aslinya saat di-deploy ke produksi, menghindari penanganan pra-pemrosesan terpisah di aplikasi klien.

        Contoh penyimpanan SavedModel menggunakan `tf.saved_model.save()`:
        ```python
        model_version = "0001"
        model_name = "my_mnist_model"
        model_path = os.path.join(model_name, model_version)
        tf.saved_model.save(model, model_path)
        ```
        Atau menggunakan metode `save()` model Keras:
        ```python
        model.save(model_path) # Akan menyimpan sebagai SavedModel jika ekstensi bukan .h5
        ```
        SavedModel dapat diperiksa menggunakan `saved_model_cli` tool. Ini menunjukkan metagraphs (computation graph plus signature definitions).

    * **TensorFlow Serving (TF Serving)**
        TF Serving adalah server model yang efisien dan tangguh yang ditulis dalam C++. Fitur utamanya meliputi kemampuan untuk mempertahankan beban tinggi, melayani beberapa versi model, dan secara otomatis men-deploy versi terbaru dengan memantau repositori model.

        **Deployment TF Serving dengan Docker:**
        Pendekatan yang direkomendasikan untuk menginstal TF Serving adalah menggunakan Docker image.

        ```bash
        docker pull tensorflow/serving
        docker run -it -rm -p 8500:8500 -p 8501:8501 \
           -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
           -e MODEL_NAME=my_mnist_model \
           tensorflow/serving
        ```
        Perintah ini menjalankan TF Serving:
        * `-it`: Membuat kontainer interaktif dan menampilkan output server.
        * `--rm`: Menghapus kontainer saat dihentikan.
        * `-p 8500:8500`: Meneruskan port TCP 8500 host ke port TCP 8500 kontainer (untuk gRPC API).
        * `-p 8501:8501`: Meneruskan port TCP 8501 host ke port TCP 8501 kontainer (untuk REST API).
        * `-v "$ML_PATH/my_mnist_model:/models/my_mnist_model"`: Membuat direktori model host tersedia di dalam kontainer.
        * `-e MODEL_NAME=my_mnist_model`: Mengatur variabel lingkungan `MODEL_NAME` di kontainer agar TF Serving tahu model mana yang akan dilayani.

        **Querying TF Serving:**
        Model yang dilayani oleh TF Serving dapat di-query melalui REST API atau gRPC API.
        * **REST API:** Menggunakan permintaan HTTP POST dengan data JSON.
            ```python
            import requests
            import json

            SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            input_data_json = json.dumps({"signature_name": "serving_default", "instances": X_new.tolist()})
            response = requests.post(SERVER_URL, data=input_data_json)
            response.raise_for_status() # raise an exception in case of error
            predictions = response.json()["predictions"]
            ```
            REST API sederhana dan cocok untuk data input/output yang tidak terlalu besar. Namun, karena berbasis teks (JSON), ini bisa menjadi tidak efisien dalam hal waktu serialisasi/deserialisasi dan ukuran payload untuk array NumPy besar.
        * **gRPC API:** Lebih efisien untuk transfer data dalam jumlah besar karena menggunakan format biner kompak dan protokol komunikasi efisien (berbasis HTTP/2 framing). Ini membutuhkan input `PredictRequest` protocol buffer dan menghasilkan `PredictResponse` protocol buffer.
            ```python
            import grpc
            from tensorflow_serving.apis.predict_pb2 import PredictRequest
            from tensorflow_serving.apis import prediction_service_pb2_grpc
            import tensorflow as tf
            import numpy as np

            request = PredictRequest()
            request.model_spec.name = model_name
            request.model_spec.signature_name = "serving_default"
            input_name = model.input_names[0]
            request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

            channel = grpc.insecure_channel('localhost:8500')
            predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
            response = predict_service.Predict(request, timeout=10.0)

            output_name = model.output_names[0]
            outputs_proto = response.outputs[output_name]
            y_proba = tf.make_ndarray(outputs_proto)
            ```
            G_RPC API lebih kompleks tetapi lebih efisien untuk transfer data besar.

        **Deploying New Model Version (Mendeploy Versi Model Baru):**
        TF Serving secara otomatis memeriksa versi model baru pada interval reguler. Ketika versi baru ditemukan (misalnya, diekspor ke direktori `0002` di samping `0001`), TF Serving akan menangani transisi dengan anggun. Secara default, ia akan menyelesaikan permintaan yang tertunda dengan versi model sebelumnya, sementara menangani permintaan baru dengan versi baru. Setelah semua permintaan tertunda dijawab, versi sebelumnya akan di-unload. Ini memastikan transisi yang mulus.

        **Automatic Batching Capability (Kemampuan Batching Otomatis):**
        TF Serving memiliki kemampuan batching otomatis yang dapat diaktifkan dengan opsi `--enable_batching` saat startup. Ini secara otomatis mengelompokkan beberapa permintaan yang diterima dalam waktu singkat sebelum menggunakannya dengan model, memberikan peningkatan kinerja yang signifikan dengan memanfaatkan kekuatan GPU.

* **Deploying a Model to a Mobile or Embedded Device (Mendeploy Model ke Perangkat Seluler atau Embedded)**
    Untuk perangkat seluler atau embedded, model perlu dioptimalkan agar ringan dan efisien. TensorFlow Lite (TFLite) menyediakan alat untuk tujuan ini.
    * **Reduced Model Size (Ukuran Model yang Dikurangi):** TFLite converter dapat mengambil SavedModel dan mengompresnya ke format yang jauh lebih ringan berdasarkan FlatBuffers. FlatBuffers dirancang untuk dimuat langsung ke RAM tanpa pra-pemrosesan, mengurangi waktu muat dan jejak memori.
        ```python
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
        tflite_model = converter.convert()
        with open("converted_model.tflite", "wb") as f:
            f.write(tflite_model)
        ```
    * **Optimizations (Optimasi):** Converter juga mengoptimalkan model dengan membuang operasi yang tidak diperlukan untuk prediksi (misalnya, operasi pelatihan) dan mengoptimalkan komputasi (misalnya, $3xa+4\times a+5\times a$ menjadi $(3+4+5)\times a$). Ini juga mencoba menggabungkan operasi seperti lapisan Batch Normalization ke dalam operasi lapisan sebelumnya.
    * **Quantization (Kuantisasi):** Mengurangi bit-width (misalnya, dari 32-bit floats ke 16-bit half-floats atau 8-bit integers) dapat secara signifikan mengurangi ukuran model.
        * **Post-training quantization (Kuantisasi pasca-pelatihan):** Menguantisasi bobot setelah pelatihan. Ini secara simetris memetakan rentang floating-point ke rentang integer 8-bit (misalnya, -m hingga +m dipetakan ke -127 hingga +127).
            ```python
            converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
            ```
            Meskipun mengurangi ukuran model, metode ini tidak mengurangi penggunaan RAM atau kecepatan komputasi saat runtime karena bobot dikembalikan ke float.
        * **Full quantization (Kuantisasi penuh):** Menguantisasi bobot dan activations sehingga komputasi dapat dilakukan sepenuhnya dengan integer, menghilangkan kebutuhan untuk operasi floating-point. Ini menghasilkan kecepatan yang jauh lebih besar dan konsumsi daya yang lebih rendah. Ini membutuhkan langkah kalibrasi menggunakan sampel data pelatihan.
        * **Quantization-aware training (Pelatihan yang sadar kuantisasi):** Digunakan jika penurunan akurasi akibat kuantisasi terlalu parah. Ini melibatkan penambahan operasi kuantisasi palsu ke model selama pelatihan sehingga model belajar mengabaikan noise kuantisasi, membuat bobot akhir lebih kuat terhadap kuantisasi.

* **TensorFlow in the Browser (TensorFlow di Browser)**
    Model dapat digunakan langsung di browser pengguna menggunakan pustaka TensorFlow.js JavaScript. Ini berguna untuk:
    * Aplikasi web dengan konektivitas intermiten atau lambat.
    * Membutuhkan respons model secepat mungkin (mengurangi latensi server).
    * Melindungi privasi pengguna dengan membuat prediksi pada sisi klien.
    `tensorflowjs_converter` tool dapat mengkonversi SavedModel atau file model Keras ke format TensorFlow.js Layers, yang dioptimalkan untuk pengunduhan web yang efisien.

* **Using GPUs to Speed Up Computations (Menggunakan GPU untuk Mempercepat Komputasi)**
    Pelatihan large neural network pada satu CPU bisa memakan waktu berhari-hari atau berminggu-minggu. GPU secara signifikan dapat mempercepat proses ini, dari hari/minggu menjadi menit/jam. Seringkali, penambahan kartu GPU ke satu mesin sudah cukup, dan menggunakan banyak mesin tidak selalu diperlukan karena keterlambatan komunikasi jaringan.

    * **Nvidia CUDA dan cuDNN:** TensorFlow mendukung kartu Nvidia dengan CUDA Compute Capability 3.5+. Penggunaan GPU memerlukan instalasi driver Nvidia, CUDA (untuk komputasi GPU umum), dan cuDNN (pustaka yang dipercepat GPU untuk DNNs).
    * **GPU VM di Cloud:** Platform cloud besar menawarkan VM GPU yang sudah dikonfigurasi sebelumnya. Google Cloud Platform (GCP) memiliki kuota GPU yang perlu ditingkatkan sebelum digunakan. Deep Learning VM Images AI Platform dapat membuat VM GPU.
    * **Colaboratory (Colab):** Cara termudah dan termurah untuk mengakses VM GPU adalah menggunakan Colab, yang gratis. Colab menyediakan Jupyter notebook yang berjalan di VM Google, dengan opsi untuk menggunakan GPU atau TPU.

    * **Managing GPU RAM (Mengelola RAM GPU):**
        Secara default, TensorFlow mengambil semua RAM di GPU yang tersedia saat komputasi pertama dijalankan untuk membatasi fragmentasi RAM GPU.
        * **Menetapkan GPU untuk setiap proses:** Gunakan variabel lingkungan `CUDA_VISIBLE_DEVICES` untuk memastikan setiap proses hanya melihat kartu GPU yang sesuai.
            ```bash
            CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
            CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_2.py
            ```
        * **Membatasi jumlah RAM GPU:** Beri tahu TensorFlow untuk hanya mengambil sejumlah RAM GPU tertentu dengan membuat perangkat GPU virtual (logical GPU device).
            ```python
            for gpu in tf.config.experimental.list_physical_devices("GPU"):
                tf.config.experimental.set_virtual_device_configuration(
                    gpu,
                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
                )
            ```
        * **Mengambil memori sesuai kebutuhan:** Atur `tf.config.experimental.set_memory_growth(gpu, True)` atau variabel lingkungan `TF_FORCE_GPU_ALLOW_GROWTH` ke `true`. Dengan opsi ini, TensorFlow hanya akan mengambil memori GPU saat dibutuhkan.

    * **Placing Operations and Variables on Devices (Menempatkan Operasi dan Variabel pada Perangkat):**
        Secara default, variabel dan operasi ditempatkan pada GPU pertama (`/gpu:0`), kecuali jika tidak ada kernel GPU yang tersedia (maka ditempatkan pada CPU, `/cpu:0`). Penempatan manual dapat dilakukan dengan `tf.device()` context.
        ```python
        with tf.device("/cpu:0"):
            c = tf.Variable(42.0)
        print(c.device) # Output: '/job:localhost/replica:0/task:0/device:CPU:0'
        ```
        `tf.config.set_soft_device_placement(True)` dapat digunakan agar TensorFlow secara otomatis beralih ke CPU jika penempatan yang diminta gagal.

    * **Parallel Execution Across Multiple Devices (Eksekusi Paralel Lintas Beberapa Perangkat):**
        TensorFlow menganalisis computation graph dari TF Function, mengidentifikasi operasi tanpa dependensi (source operations), dan menambahkannya ke antrean evaluasi perangkat masing-masing. Operasi CPU dijalankan secara paralel di kumpulan thread "inter-op". Operasi GPU dievaluasi secara berurutan, tetapi sebagian besar kernel GPU adalah multithreaded dan memanfaatkan banyak thread GPU.

* **Training Models Across Multiple Devices (Melatih Model Lintas Beberapa Perangkat)**
    Dua pendekatan utama untuk melatih model tunggal di beberapa perangkat:
    * **Model Parallelism (Paralelisme Model): Model dibagi di antara perangkat.** Ini rumit dan sangat bergantung pada arsitektur jaringan saraf. Misalnya, menempatkan setiap lapisan pada perangkat yang berbeda tidak efisien karena setiap lapisan harus menunggu output dari lapisan sebelumnya. Pemisahan vertikal dapat menyebabkan banyak komunikasi antar-perangkat. Beberapa arsitektur seperti CNN (lapisan yang hanya terhubung sebagian) atau RNN (deep recurrent neural networks) lebih mudah didistribusikan secara efisien.
    * **Data Parallelism (Paralelisme Data): Model direplikasi di setiap perangkat, dan setiap replika dilatih pada subset data yang berbeda.** Gradien yang dihitung oleh setiap replika kemudian dirata-ratakan dan digunakan untuk memperbarui parameter model. Ini umumnya lebih sederhana dan lebih efisien.

        * **Mirrored Strategy (Strategi Mirrored):** Semua parameter model sepenuhnya dicerminkan di semua GPU, dan pembaruan parameter yang sama diterapkan pada setiap GPU. Semua replika tetap identik. Bagian sulitnya adalah menghitung rata-rata gradien dari semua GPU dan mendistribusikan hasilnya secara efisien, yang dilakukan menggunakan algoritma AllReduce. Ini sangat efisien, terutama pada satu mesin.
            ```python
            distribution = tf.distribute.MirroredStrategy()
            with distribution.scope():
                mirrored_model = keras.models.Sequential([...])
                mirrored_model.compile([...])
            batch_size = 100 # must be divisible by the number of replicas
            history = mirrored_model.fit(X_train, y_train, epochs=10, batch_size=batch_size)
            ```
            `tf.keras` secara otomatis mereplikasi variabel dan operasi di semua perangkat GPU yang tersedia dalam konteks `MirroredStrategy`.

        * **Centralized Parameters (Parameter Terpusat):** Parameter model disimpan di luar perangkat GPU pekerja, misalnya di CPU atau pada server parameter khusus dalam setup terdistribusi.
            * **Synchronous updates (Pembaruan sinkron):** Aggregator menunggu sampai semua gradien tersedia sebelum menghitung rata-rata dan memperbarui parameter. Replika menunggu pembaruan parameter. Kelemahannya adalah perangkat yang lebih lambat dapat menahan perangkat lain.
            * **Asynchronous updates (Pembaruan asinkron):** Setiap replika langsung menggunakan gradien yang dihitung untuk memperbarui parameter tanpa agregasi atau sinkronisasi. Ini lebih cepat dalam hal langkah pelatihan per menit. Namun, gradien bisa menjadi "stale" (kadaluwarsa) karena parameter telah diperbarui oleh replika lain, yang dapat memperlambat konvergensi atau bahkan menyebabkan divergensi.

        * **Bandwidth saturation (Saturasi Bandwidth):** Penambahan GPU dapat mencapai titik di mana waktu yang dihabiskan untuk memindahkan data melebihi peningkatan kecepatan komputasi. Ini lebih parah untuk model dense besar dan kurang parah untuk model kecil atau model sparse. Strateginya adalah menggunakan beberapa GPU yang kuat dan mengelompokkan GPU pada server yang saling terhubung dengan baik. Mengurangi presisi float dari 32 bit menjadi 16 bit juga dapat mengurangi transfer data. Sharding parameter di beberapa parameter server juga membantu mengurangi beban jaringan.

    * **Distribution Strategies API:** TensorFlow menyediakan API yang menyederhanakan kompleksitas pelatihan terdistribusi.
        * `tf.distribute.MirroredStrategy()`: Untuk data parallelism dengan mirrored strategy pada satu mesin.
        * `tf.distribute.experimental.MultiWorkerMirroredStrategy()`: Untuk data parallelism dengan mirrored strategy di beberapa mesin (cluster).
        * `tf.distribute.experimental.CentralStorageStrategy()`: Untuk data parallelism dengan centralized parameters.
        * `tf.distribute.experimental.TPUStrategy()`: Untuk melatih model di TPUs.

* **Training a Model on a TensorFlow Cluster (Melatih Model pada Cluster TensorFlow)**
    Cluster TensorFlow adalah sekelompok proses TensorFlow yang berjalan paralel, biasanya di mesin yang berbeda. Setiap proses disebut "task" atau "TF server" dengan peran seperti "worker", "chief", "ps" (parameter server), atau "evaluator". Spesifikasi cluster (IP, port, dan jenis setiap task) harus ditentukan, seringkali melalui variabel lingkungan `TF_CONFIG`.

* **Running Large Training Jobs on Google Cloud AI Platform (Menjalankan Pekerjaan Pelatihan Besar di Google Cloud AI Platform)**
    Google AI Platform memungkinkan deployment pekerjaan pelatihan dengan kode pelatihan yang sama seperti yang dijalankan pada cluster TF sendiri. Platform ini akan menyediakan dan mengkonfigurasi VM GPU yang diperlukan.
    * **`gcloud` command-line tool:** Digunakan untuk memulai pekerjaan pelatihan.
        ```bash
        gcloud ai-platform jobs submit training my_job_20190531_164700 \
            --region asia-southeast1 \
            --scale-tier PREMIUM_1 \
            --runtime-version 2.0 \
            --python-version 3.5 \
            --package-path /my_project/src/trainer \
            --module-name trainer.task \
            --staging-bucket gs://my-staging-bucket \
            --job-dir gs://my-mnist-model-bucket/trained_model \
            --my-extra-argument1 foo --my-extra-argument2 bar
        ```
        Perintah ini menentukan nama pekerjaan, wilayah, tingkat skala (misalnya, PREMIUM_1 untuk 20 pekerja dan 11 parameter server). Ini juga menentukan versi runtime, versi Python, jalur kode pelatihan, modul, bucket staging, dan direktori pekerjaan.

    * **Black Box Hyperparameter Tuning on AI Platform (Penyetelan Hyperparameter Black Box di AI Platform):**
        AI Platform menyediakan layanan penyetelan hyperparameter Bayesian optimization yang disebut Google Vizier. Ini digunakan dengan meneruskan file konfigurasi YAML saat membuat pekerjaan.
        ```yaml
        trainingInput:
          hyperparameters:
            goal: MAXIMIZE
            hyperparameterMetricTag: accuracy
            maxTrials: 10
            maxParallelTrials: 2
            params:
            - parameterName: n_layers
              type: INTEGER
              minValue: 10
              maxValue: 100
              scaleType: UNIT_LINEAR_SCALE
            - parameterName: momentum
              type: DOUBLE
              minValue: 0.1
              maxValue: 1.0
              scaleType: UNIT_LOG_SCALE
        ```
        Ini menginstruksikan AI Platform untuk memaksimalkan metrik "accuracy" dengan menjalankan sejumlah trial secara paralel, dan menyetel hyperparameter seperti `n_layers` dan `momentum` dalam rentang yang ditentukan. Kode pelatihan harus melaporkan metrik ini kembali ke AI Platform, yang dapat dilakukan dengan menggunakan callback `TensorBoard()`.